# Setup do projeto

In [7]:
# Comandos para instalar cada biblioteca caso não esteja sendo usado um ambiente virtual com elas instaladas
# !pip install geopy
# !pip install pandas
# !pip install numpy
# !pip install seaborn
# !pip install requests
# !pip install unicodedata
# !pip install matplotlib.pyplot
# !pip install kagglehub

In [ ]:
#Importando bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import requests
import unicodedata
import matplotlib.pyplot as plt
import geopy

In [ ]:
#Importando arquivos da API do Kaggle

import kagglehub

path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\itall\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2


In [ ]:
#Lendo cada arquivo importado do Kaggle

customers = pd.read_csv(path+'/olist_customers_dataset.csv')
geolocation = pd.read_csv(path+'/olist_geolocation_dataset.csv')
order_items = pd.read_csv(path+'/olist_order_items_dataset.csv')
orders_payments = pd.read_csv(path+'/olist_order_payments_dataset.csv')
orders_reviews = pd.read_csv(path+'/olist_order_reviews_dataset.csv')
orders = pd.read_csv(path+'/olist_orders_dataset.csv')
products = pd.read_csv(path+'/olist_products_dataset.csv')
sellers = pd.read_csv(path+'/olist_sellers_dataset.csv')
category_name = pd.read_csv(path+'/product_category_name_translation.csv')

In [ ]:
#Para simplificar o projeto, as funções de validacoes foram colocadas em arquivos separados como funções python

import Validations.validations as validations
import Validations.validate_customers as validate_customers
import Validations.validate_geolocation as validate_geolocation
import Validations.validate_orders_items as validate_orders_items
import Validations.validate_orders_payments as validate_orders_payments
import Validations.validate_orders_reviews as validate_orders_reviews
import Validations.validate_orders as validate_orders
import Validations.validate_orders_reviews as validate_orders_reviews
import Validations.validate_products as validate_products
import Validations.validate_sellers as validate_sellers

![Brazilian E-Commerce Public Dataset by Olist Data Schema](./Images/Brazilian_E-Commerce_Public_Dataset_by_Olist_Data_Schema.png)

As tabelas e seus relacionamentos estão em um padrão levemente direcionado ao Snow Flake. A estrutura não será alterada pois a velocidade de consulta não é tão relevante e como são poucos dados, mesmo no modelo SnowFlake não fica tão complexo

## Análise Inicial e Exploratória - Orders

In [ ]:
orders.info()
orders.head()

In [ ]:
registros_invalidos_orders = validate_orders.validate_orders(orders)

Há três colunas com valores inválido:
*   order_approved_at;
*   order_delivered_carrier_date;
*   order_delivered_customer_date

A falta desses dados pode dever-se ao momento que as informações foram extraídas. Ou seja, quando o recorte dos pedidos foi tirado, alguns pedidos não foram enviados, outros não chegaram. Além disso, pedidos cancelados também não tem data de entrega por não terem sido entregues. Mesmo assim é possível ver que alguns pedidos têm alguma data inválida e estão com o status "entregue", mas são pouquissímos.

Levando em consideração que todos os registros com alguma data inválida são cerca de 3% da amostra e que não é a intenção analisar no momento taxa de cancelamento, pedidos em andamentos, entre outros, foi decidido retirar os registros com alguma data inválida. A exclusão será feita posteriormente, os dados das linhas inválidas serão guardados

##Análise Inicial e Exploratória - Orders Reviews

In [ ]:
orders_reviews.info
orders_reviews.head()

###Análise de colunas inválidas - Orders Reviews

Não houveram valores inválidos para este dataset. Portanto, não há itens para serem retirados aqui




###Hipóteses - Orders Reviews

#### Hipótese 01: A data das colunas review_creation_date e review_answer_timestamp são sempre iguais

Olhando para os nomes das colunas e pelo senso comum, é possível intuir que a pessoa tenha feito o review e o comentário no mesmo momento

In [467]:
#Verifica se os dias apontados nas colunas review_answer_timestamp e review_creation_date são iguais
orders_reviews_datas_coincidem = orders_reviews.copy()

#Converte as colunas para o tipo datetime para poderem ser comparadas posteriormente por dt.date
orders_reviews_datas_coincidem['review_creation_date'] = pd.to_datetime(orders_reviews_datas_coincidem['review_creation_date'], errors='coerce')
orders_reviews_datas_coincidem['review_answer_timestamp'] = pd.to_datetime(orders_reviews_datas_coincidem['review_answer_timestamp'], errors='coerce')

#Compara as colunas com dt.date para saber se a apenas a data (e não data e horário) são iguais
orders_reviews_datas_coincidem['datas_coincidem'] = orders_reviews_datas_coincidem['review_creation_date'].dt.date == orders_reviews_datas_coincidem['review_answer_timestamp'].dt.date

#Soma a quantidade de dados que coincidem e não coincidem
contagem_datas_coincidem = orders_reviews_datas_coincidem['datas_coincidem'].sum()
contagem_datas_nao_coincidem = len(orders_reviews_datas_coincidem) - contagem_datas_coincidem

#Mostra resultado na tela
print(f"Nº de registros com review_creation_date e review_answer_timestamp iguais: {contagem_datas_coincidem} de {len(orders_reviews_datas_coincidem)} ({contagem_datas_coincidem/len(orders_reviews_datas_coincidem)*100:.2f}%)" )
print(f"Nº de registros com review_creation_date e review_answer_timestamp diferentes: {contagem_datas_nao_coincidem} de {len(orders_reviews_datas_coincidem)}({contagem_datas_nao_coincidem/len(orders_reviews_datas_coincidem)*100:.2f}%)")


Nº de registros com review_creation_date e review_answer_timestamp iguais: 24361 de 99224 (24.55%)
Nº de registros com review_creation_date e review_answer_timestamp diferentes: 74863 de 99224(75.45%)


**Resultado:** Hipótese inválida. Como visto no resultado, a maioria das reviews tiveram uma data de comentário e uma data review diferentes. Com isto surge a dúvida, por quê as datas são diferentes?

##Análise Inicial e Exploratória - Orders Payments

In [ ]:
orders_payments.info()
orders_payments.head()

###Análise de colunas inválidas - Orders Payments


## Análise de colunas inválidas - Orders Items

In [ ]:
order_items.info()
order_items.head()

### Análise de colunas inválidas - Orders Items


In [470]:
products.info()
products.describe()
products.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


##Análise Inicial e Exploratória - Products

###Análise de colunas inválidas - Products

Há alguns produtos inválidos por não terem categoria e nome e produtos inválidos por não terem especificado características como peso, altura, etc. Como apenas a categoria importa para o tipo de análise que irá ser feita, apenas os itens com categorias inválidas fora classificados como inválidos

##Análise Inicial e Exploratória - Sellers

In [471]:
sellers.info()
sellers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


###Análise de colunas inválidas - Seller

#Respostas de Negócio

## Quantos pedidos foram entregues dentro do prazo em números absolutos e em porcentagem?


In [472]:

orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [473]:
#Copying just the necessary data
orders_delivery_time = orders[['order_delivered_customer_date', 'order_estimated_delivery_date']].copy()

#Converting the columns to Datetime and extracting just the date in order_delivered_customer_date case
orders_delivery_time['order_delivered_customer_date'] = pd.to_datetime(orders_delivery_time['order_delivered_customer_date'], errors='coerce').dt.date
orders_delivery_time['order_estimated_delivery_date'] = pd.to_datetime(orders_delivery_time['order_estimated_delivery_date'], errors='coerce')

#Excluding invalid records
orders_delivery_time.dropna(subset=['order_delivered_customer_date', 'order_estimated_delivery_date'])

#Comparing the delivered date with estimeted delivery date
orders_delivery_time['delivered_on_date'] = orders_delivery_time['order_delivered_customer_date'] <= orders_delivery_time['order_estimated_delivery_date']

#Calculating results
orders_total_number = len(orders_delivery_time)
orders_deliverd_on_time = orders_delivery_time['delivered_on_date'].sum()
orders_deliverd_not_on_time = len(orders_delivery_time) - orders_delivery_time['delivered_on_date'].sum()
orders_deliverd_on_time_percentage = round(orders_deliverd_on_time / orders_total_number * 100, 2)
orders_deliverd_not_on_time_percentage = round(orders_deliverd_not_on_time / orders_total_number * 100, 2)

#Showing results
print(f'Total number of orders delivered on date: {orders_deliverd_on_time} of {orders_total_number} ({orders_deliverd_on_time_percentage}%) ')
print(f'Total number of orders not delivered on date: {orders_deliverd_not_on_time} of {orders_total_number} ({orders_deliverd_not_on_time_percentage}%) ')


Total number of orders delivered on date: 89941 of 99441 (90.45%) 
Total number of orders not delivered on date: 9500 of 99441 (9.55%) 


## Qual a distância média que um pedido se move em km por dia?

In [474]:
#Copying just the necessary data
orders_distance_per_day = orders[['order_id', 'customer_id', 'order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date']].copy()

#Converting the columns to Datetime and extracting just the date in order_delivered_customer_date case
orders_distance_per_day['order_delivered_customer_date'] = pd.to_datetime(orders_distance_per_day['order_delivered_customer_date'], errors='coerce').dt.date
orders_delivery_time['order_estimated_delivery_date'] = pd.to_datetime(orders_delivery_time['order_estimated_delivery_date'], errors='coerce').dt.date
orders_distance_per_day['order_delivered_carrier_date'] = pd.to_datetime(orders_distance_per_day['order_delivered_carrier_date'], errors='coerce').dt.date

orders_distance_per_day.head()

,order_id,customer_id,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-18 00:00:00,2017-10-10,2017-10-04
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-08-13 00:00:00,2018-08-07,2018-07-26
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-09-04 00:00:00,2018-08-17,2018-08-08
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-12-15 00:00:00,2017-12-02,2017-11-22
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-26 00:00:00,2018-02-16,2018-02-14


In [475]:
orders_distance_per_day = pd.merge(order_items, orders_distance_per_day, on='order_id', how='left')
orders_distance_per_day = orders_distance_per_day[['order_id', 'seller_id', 'customer_id','order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date']]
orders_distance_per_day.head()

,order_id,seller_id,customer_id,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,2017-09-29 00:00:00,2017-09-20,2017-09-19
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,2017-05-15 00:00:00,2017-05-12,2017-05-04
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,2018-02-05 00:00:00,2018-01-22,2018-01-16
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,2018-08-20 00:00:00,2018-08-14,2018-08-10
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,2017-03-17 00:00:00,2017-03-01,2017-02-16


In [476]:
orders_distance_per_day = pd.merge(orders_distance_per_day, customers, on='customer_id', how='left')
orders_distance_per_day = orders_distance_per_day[['order_id', 'seller_id', 'customer_id', 'customer_city','order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date']]
orders_distance_per_day.head()

,order_id,seller_id,customer_id,customer_city,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,campos dos goytacazes,2017-09-29 00:00:00,2017-09-20,2017-09-19
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,santa fe do sul,2017-05-15 00:00:00,2017-05-12,2017-05-04
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,para de minas,2018-02-05 00:00:00,2018-01-22,2018-01-16
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,atibaia,2018-08-20 00:00:00,2018-08-14,2018-08-10
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,varzea paulista,2017-03-17 00:00:00,2017-03-01,2017-02-16


In [477]:
geolocation_merge = geolocation.drop_duplicates(subset=['geolocation_city'], keep='first').copy()

In [478]:
orders_distance_per_day = pd.merge(orders_distance_per_day, geolocation_merge, left_on='customer_city', right_on='geolocation_city', how='left')
orders_distance_per_day = orders_distance_per_day.rename(columns={
    'geolocation_lat': 'customer_geolocation_lat',
    'geolocation_lng': 'customer_geolocation_lng',
})
orders_distance_per_day = orders_distance_per_day[['order_id', 'seller_id', 'customer_id', 'customer_city','order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date', 'customer_geolocation_lat', 'customer_geolocation_lng']]

orders_distance_per_day.head()


,order_id,seller_id,customer_id,customer_city,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,customer_geolocation_lat,customer_geolocation_lng
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,campos dos goytacazes,2017-09-29 00:00:00,2017-09-20,2017-09-19,-21.720140,-41.310453
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,santa fe do sul,2017-05-15 00:00:00,2017-05-12,2017-05-04,-20.212393,-50.941471
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,para de minas,2018-02-05 00:00:00,2018-01-22,2018-01-16,-19.847170,-44.601957
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,atibaia,2018-08-20 00:00:00,2018-08-14,2018-08-10,-23.112774,-46.548885
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,varzea paulista,2017-03-17 00:00:00,2017-03-01,2017-02-16,-23.184985,-46.825698


In [479]:
orders_distance_per_day = pd.merge(orders_distance_per_day, sellers, on='seller_id', how='left')
orders_distance_per_day = orders_distance_per_day[['order_id', 'seller_id', 'seller_city','customer_id', 'customer_city','order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date', 'customer_geolocation_lat', 'customer_geolocation_lng']]

orders_distance_per_day.head()


,order_id,seller_id,seller_city,customer_id,customer_city,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,customer_geolocation_lat,customer_geolocation_lng
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,volta redonda,3ce436f183e68e07877b285a838db11a,campos dos goytacazes,2017-09-29 00:00:00,2017-09-20,2017-09-19,-21.720140,-41.310453
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,sao paulo,f6dd3ec061db4e3987629fe6b26e5cce,santa fe do sul,2017-05-15 00:00:00,2017-05-12,2017-05-04,-20.212393,-50.941471
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,borda da mata,6489ae5e4333f3693df5ad4372dab6d3,para de minas,2018-02-05 00:00:00,2018-01-22,2018-01-16,-19.847170,-44.601957
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,franca,d4eb9395c8c0431ee92fce09860c5a06,atibaia,2018-08-20 00:00:00,2018-08-14,2018-08-10,-23.112774,-46.548885
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,loanda,58dbd0b2d70206bf40e62cd34e84d795,varzea paulista,2017-03-17 00:00:00,2017-03-01,2017-02-16,-23.184985,-46.825698


In [480]:
orders_distance_per_day = pd.merge(orders_distance_per_day, geolocation_merge, left_on='seller_city', right_on='geolocation_city', how='left')
orders_distance_per_day = orders_distance_per_day.rename(columns={
    'geolocation_lat': 'seller_geolocation_lat',
    'geolocation_lng': 'seller_geolocation_lng',
})
orders_distance_per_day = orders_distance_per_day[['order_id', 'seller_id', 'customer_id', 'customer_city','order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date', 'customer_geolocation_lat', 'customer_geolocation_lng', 'seller_geolocation_lat', 'seller_geolocation_lng']]

orders_distance_per_day.head()

,order_id,seller_id,customer_id,customer_city,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,customer_geolocation_lat,customer_geolocation_lng,seller_geolocation_lat,seller_geolocation_lng
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,campos dos goytacazes,2017-09-29 00:00:00,2017-09-20,2017-09-19,-21.720140,-41.310453,-22.529850,-44.124115
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,santa fe do sul,2017-05-15 00:00:00,2017-05-12,2017-05-04,-20.212393,-50.941471,-23.545621,-46.639292
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,para de minas,2018-02-05 00:00:00,2018-01-22,2018-01-16,-19.847170,-44.601957,-22.264094,-46.158564
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,atibaia,2018-08-20 00:00:00,2018-08-14,2018-08-10,-23.112774,-46.548885,-20.531935,-47.399447
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,varzea paulista,2017-03-17 00:00:00,2017-03-01,2017-02-16,-23.184985,-46.825698,-22.931427,-53.133759


In [481]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

def calcular_distancia(row):
    try:
        coords_cliente = (row['customer_geolocation_lat'], row['customer_geolocation_lng'])
        coords_vendedor = (row['seller_geolocation_lat'], row['seller_geolocation_lng'])

        distancia = geodesic(coords_cliente, coords_vendedor).kilometers
        return distancia
    except (ValueError, TypeError):
        return np.nan

orders_distance_per_day['distance_km'] = orders_distance_per_day.apply(calcular_distancia, axis=1)


In [482]:
orders_distance_per_day = orders_distance_per_day[['order_id', 'seller_id', 'customer_id','order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date', 'distance_km']]
orders_distance_per_day.head()

,order_id,seller_id,customer_id,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,distance_km
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,2017-09-29 00:00:00,2017-09-20,2017-09-19,303.813291
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,2017-05-15 00:00:00,2017-05-12,2017-05-04,577.773431
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,2018-02-05 00:00:00,2018-01-22,2018-01-16,312.690412
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,2018-08-20 00:00:00,2018-08-14,2018-08-10,298.992802
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,2017-03-17 00:00:00,2017-03-01,2017-02-16,646.999551


In [483]:
orders_distance_per_day['delivery_in_days'] = (orders_distance_per_day['order_delivered_customer_date'] - orders_distance_per_day['order_delivered_carrier_date'])
orders_distance_per_day['delivery_in_days'] = pd.to_timedelta(orders_distance_per_day['delivery_in_days']).dt.days
orders_distance_per_day.head()

,order_id,seller_id,customer_id,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,distance_km,delivery_in_days
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,2017-09-29 00:00:00,2017-09-20,2017-09-19,303.813291,1.0
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,2017-05-15 00:00:00,2017-05-12,2017-05-04,577.773431,8.0
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,2018-02-05 00:00:00,2018-01-22,2018-01-16,312.690412,6.0
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,2018-08-20 00:00:00,2018-08-14,2018-08-10,298.992802,4.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,2017-03-17 00:00:00,2017-03-01,2017-02-16,646.999551,13.0


In [484]:
orders_distance_per_day['average_distance_in_days'] = (orders_distance_per_day['distance_km']/orders_distance_per_day['delivery_in_days'])
orders_distance_per_day.head()

,order_id,seller_id,customer_id,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,distance_km,delivery_in_days,average_distance_in_days
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,2017-09-29 00:00:00,2017-09-20,2017-09-19,303.813291,1.0,303.813291
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,2017-05-15 00:00:00,2017-05-12,2017-05-04,577.773431,8.0,72.221679
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,2018-02-05 00:00:00,2018-01-22,2018-01-16,312.690412,6.0,52.115069
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,2018-08-20 00:00:00,2018-08-14,2018-08-10,298.992802,4.0,74.748200
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,2017-03-17 00:00:00,2017-03-01,2017-02-16,646.999551,13.0,49.769196


In [485]:
orders_distance_per_day['difference_days_estimed_delivered_date'] = pd.to_datetime(orders_distance_per_day['order_estimated_delivery_date']) - pd.to_datetime(orders_distance_per_day['order_delivered_carrier_date'])
orders_distance_per_day

,order_id,seller_id,customer_id,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,distance_km,delivery_in_days,average_distance_in_days,difference_days_estimed_delivered_date
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,2017-09-29 00:00:00,2017-09-20,2017-09-19,303.813291,1.0,303.813291,10 days
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,2017-05-15 00:00:00,2017-05-12,2017-05-04,577.773431,8.0,72.221679,11 days
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,2018-02-05 00:00:00,2018-01-22,2018-01-16,312.690412,6.0,52.115069,20 days
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,2018-08-20 00:00:00,2018-08-14,2018-08-10,298.992802,4.0,74.748200,10 days
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,2017-03-17 00:00:00,2017-03-01,2017-02-16,646.999551,13.0,49.769196,29 days
...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,b8bc237ba3788b23da09c0f1f3a3288c,b51593916b4b8e0d6f66f2ae24f2673d,2018-05-18 00:00:00,2018-05-10,2018-04-25,2737.196264,15.0,182.479751,23 days
112646,fffcd46ef2263f404302a634eb57f7eb,f3c38ab652836d21de61fb8314b69182,84c5d4fbaf120aae381fad077416eaa0,2018-08-01 00:00:00,2018-07-23,2018-07-17,339.003763,6.0,56.500627,15 days
112647,fffce4705a9662cd70adb13d4a31832d,c3cfdc648177fdbbbb35635a37472c53,29309aa813182aaddc9b259e31b870e6,2017-11-10 00:00:00,2017-10-28,2017-10-26,339.003763,2.0,169.501881,15 days
112648,fffe18544ffabc95dfada21779c9644f,2b3e4a2a3ea8e01938cabda2a3e5cc79,b5e6afd5a41800fdf401e0272ca74655,2017-08-25 00:00:00,2017-08-16,2017-08-15,68.984116,1.0,68.984116,10 days


In [486]:
orders_distance_per_day['order_estimated_delivery_date'] = pd.to_datetime(
    orders_distance_per_day['order_estimated_delivery_date']
)

dias_a_reduzir = pd.to_timedelta(
    orders_distance_per_day['delivery_in_days'] * 2,
    unit='D')

orders_distance_per_day['dias da estimativa para serem reduzidos'] = orders_distance_per_day['order_estimated_delivery_date'] - dias_a_reduzir
orders_distance_per_day.head()

,order_id,seller_id,customer_id,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,distance_km,delivery_in_days,average_distance_in_days,difference_days_estimed_delivered_date,dias da estimativa para serem reduzidos
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,2017-09-29,2017-09-20,2017-09-19,303.813291,1.0,303.813291,10 days,2017-09-27
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,2017-05-15,2017-05-12,2017-05-04,577.773431,8.0,72.221679,11 days,2017-04-29
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,2018-02-05,2018-01-22,2018-01-16,312.690412,6.0,52.115069,20 days,2018-01-24
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,2018-08-20,2018-08-14,2018-08-10,298.992802,4.0,74.748200,10 days,2018-08-12
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,2017-03-17,2017-03-01,2017-02-16,646.999551,13.0,49.769196,29 days,2017-02-19


In [487]:
orders_distance_per_day[orders_distance_per_day['difference_days_estimed_delivered_date'].dt.days < 0]

,order_id,seller_id,customer_id,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,distance_km,delivery_in_days,average_distance_in_days,difference_days_estimed_delivered_date,dias da estimativa para serem reduzidos
227,009742b43f4530bc3db7f62332c50f4b,17e34d8224d27a541263c4c64b11a56b,84dac31647902f79570207a7f29c2675,2017-11-10,2017-12-06,2017-11-28,NaN,8.0,NaN,-18 days,2017-10-25
248,00a379dfab816a83741012b71b264098,1da3aeb70d7989d1e6d9b0e887f97c23,5e3b89299a7d6e3e10f9bc6e77923f88,2018-08-08,2018-08-10,2018-08-09,0.000000,1.0,0.000000,-1 days,2018-08-06
307,00c00d5b36157595e21108ba1da2828f,391fc6631aebcf3004804e51b40bcf1e,2b27fae969b7d2bc173e466a742e50c6,2017-09-27,2017-10-04,2017-09-28,237.601342,6.0,39.600224,-1 days,2017-09-15
308,00c00d5b36157595e21108ba1da2828f,391fc6631aebcf3004804e51b40bcf1e,2b27fae969b7d2bc173e466a742e50c6,2017-09-27,2017-10-04,2017-09-28,237.601342,6.0,39.600224,-1 days,2017-09-15
360,00e054d0da011d5016f31011af488f4f,c3867b4666c7d76867627c2f7fb22e21,fe43b2f349c9ab46883ce0e02491432c,2018-03-28,2018-05-08,2018-04-27,353.898163,11.0,32.172560,-30 days,2018-03-06
...,...,...,...,...,...,...,...,...,...,...,...
111347,fcf77320278b491829c6e5d767a3f20d,54965bbe3e4f07ae045b90b0b8541f52,7537ac8265339393d7a9fc8f4cfaf361,2017-12-19,2018-01-23,2018-01-04,370.105670,19.0,19.479246,-16 days,2017-11-11
111874,fe225fc0702b1d46a931e88d60c7be25,0692610d8abe24f287e9fae90ea0bbee,c90e1fc62717b61b9e82c4ae6f32bb2d,2018-08-27,2018-08-29,2018-08-28,0.000000,1.0,0.000000,-1 days,2018-08-25
111961,fe58b25c8ebdc65c3d4cb34070bf9e5d,95b293867b5862941c9cd74f756a3c68,d70a4c877dba1679f5c7ded688e39609,2018-03-12,2018-04-02,2018-03-14,361.745680,19.0,19.039246,-2 days,2018-02-02
112120,febcb6742e4652312e996b6bc7fbcdd8,827f8f69dfa529c561901c4f2e0f332f,414473afb9a8e920051c8ecd9e3a59a3,2018-03-27,2018-04-13,2018-03-30,516.172282,14.0,36.869449,-3 days,2018-02-27


##